# Проект по SQL

**Контекст** - есть база данных конкурента. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта -  приложения для тех, кто любит читать.

**План**

- [Загрузить библиотеки, данные и посмотреть на базовую информацию по таблицам](#step1)
- [Сделать запросы](#step2):
    - [Посчитать, сколько книг вышло после 1 января 2000 года](#step2.1);
    - [Для каждой книги посчитать количество обзоров и среднюю оценку](#step2.2);
    - [Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключите из анализа брошюры](#step2.3);
    - [Определить автора с самой высокой средней оценкой книг (учитывать только книги с 50 и более оценками)](#step2.4);
    - [Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок](#step2.5).

*После запросов, сформулированных на SQL, в ячейках ниже добавлен и закомментирован код проверки запросов, написанный на Python*.

# Шаг 1. Загрузка таблиц<a id="step1"></a>   

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

Подключение к базе данных (данные подключения к базе заменены на ххх в соответствие с требованиями безопасности).

In [2]:
db_config = {'user': 'praktikum_student',
             'pwd': 'xxx',
             'host': 'xxx',
             'port': 1234,
             'db':  'data-analyst-final-project-db'}

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                   db_config['pwd'],
                                                   db_config['host'],
                                                   db_config['port'],
                                                   db_config['db'])

engine = create_engine(connection_string)

Загрузка таблиц `books`, `authors`, `publishers`, `ratings`, `reviews` и базовой информации по ним.

In [3]:
query_1 = 'SELECT * FROM books;'
books = pd.io.sql.read_sql(query_1, con = engine)

query_2 = 'SELECT * FROM authors;'
authors = pd.io.sql.read_sql(query_2, con = engine)

query_3 = 'SELECT * FROM publishers;'
publishers = pd.io.sql.read_sql(query_3, con = engine)

query_4 = 'SELECT * FROM ratings;'
ratings = pd.io.sql.read_sql(query_4, con = engine)

query_5 = 'SELECT * FROM reviews;'
reviews = pd.io.sql.read_sql(query_5, con = engine)

In [4]:
for df in [books, authors, publishers, ratings, reviews]:
    df.head()
    df.info()
    df.duplicated().sum()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
book_id             1000 non-null int64
author_id           1000 non-null int64
title               1000 non-null object
num_pages           1000 non-null int64
publication_date    1000 non-null object
publisher_id        1000 non-null int64
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


0

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
author_id    636 non-null int64
author       636 non-null object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


...

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
rating_id    6456 non-null int64
book_id      6456 non-null int64
username     6456 non-null object
rating       6456 non-null int64
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


0

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
review_id    2793 non-null int64
book_id      2793 non-null int64
username     2793 non-null object
text         2793 non-null object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


0

# Шаг 2. Запросы к базе данных<a id="step2"></a>   

## Задание 1. Сколько книг вышло после 1 января 2000 года<a id="step2.1"></a>   

In [5]:
query_task_1 = '''
                SELECT COUNT(book_id) num_of_books_after_2001_01_01 
                FROM books 
                WHERE publication_date::timestamp > '2000-01-01';
               '''
task_1 = pd.io.sql.read_sql(query_task_1, con = engine) 
task_1

,num_of_books_after_2001_01_01
0,819


In [6]:
#Код для проверки

# books['publication_date'] = pd.to_datetime(books['publication_date'])
# books[books.publication_date > '2000-01-01'].title.count()

## Задание 2. Количество обзоров и средняя оценка для каждой книги <a id="step2.2"></a>   

In [7]:
query_task_2 = '''
                SELECT 
                    books.title, 
                    books.book_id,
                    COUNT(DISTINCT(review_id)) as num_of_reviews, 
                    AVG(ratings.rating) as avg_rating
                FROM books
                LEFT JOIN reviews ON reviews.book_id = books.book_id
                LEFT JOIN ratings ON ratings.book_id = books.book_id
                GROUP BY books.title, books.book_id
                ORDER BY books.book_id;
                '''
task_2 = pd.io.sql.read_sql(query_task_2, con = engine) 
task_2

,title,book_id,num_of_reviews,avg_rating
0,'Salem's Lot,1,2,3.666667
1,1 000 Places to See Before You Die,2,1,2.500000
2,13 Little Blue Envelopes (Little Blue Envelope...,3,3,4.666667
3,1491: New Revelations of the Americas Before C...,4,2,4.500000
4,1776,5,4,4.000000
...,...,...,...,...
995,Wyrd Sisters (Discworld #6; Witches #2),996,3,3.666667
996,Xenocide (Ender's Saga #3),997,3,3.400000
997,Year of Wonders,998,4,3.200000
998,You Suck (A Love Story #2),999,2,4.500000


In [8]:
#Код для проверки
# books_reviews = reviews.groupby('book_id').agg({'review_id':'count'})
# books_ratings = ratings.groupby('book_id').agg({'rating':'mean'})
# avg_rating = books[['title', 'book_id']].merge(books_reviews, on='book_id', how='left')\
#                                         .merge(books_ratings, on='book_id', how='left')

In [9]:
# avg_rating = avg_rating.fillna(0)
# avg_rating = avg_rating.rename(columns={'review_id':'num_of_reviews', 'rating':'avg_rating'}) 
# avg_rating
# task_2.equals(avg_rating)

In [10]:
# taskBool = (task_2 != avg_rating).stack()
# taskdiff = pd.concat([task_2.stack()[taskBool], avg_rating.stack()[taskBool]], axis=1)
# taskdiff.columns=["task_2", "avg_rating"]
# taskdiff

*По факту различий между результатами нет.*

## Задание 3. Издательство, которое выпустило наибольшее число книг толще 50 страниц <a id="step2.3"></a>   

In [11]:
query_task_3 = '''
          SELECT books.publisher_id, publishers.publisher, COUNT(books.publisher_id) as num_of_books
          FROM books
          JOIN publishers on publishers.publisher_id = books.publisher_id
          WHERE num_pages > 50
          GROUP BY books.publisher_id, publishers.publisher
          ORDER BY num_of_books DESC
          LIMIT 1
          ;
          '''
task_3 = pd.io.sql.read_sql(query_task_3, con = engine) 
task_3

,publisher_id,publisher,num_of_books
0,212,Penguin Books,42


In [12]:
#Код для проверки
# books_publishers = books.query('num_pages > 50').groupby('publisher_id').agg({'title':'count'})\
#                .merge(publishers, on='publisher_id', how='left')
# books_publishers[books_publishers.title == books_publishers.title.max()]

## Задание 4. Автор с самой высокой средней оценкой книг <a id="step2.4"></a>   

(учитывать только книги с 50 и более оценками)

In [17]:
query_task_4 = '''
          SELECT author, AVG(avg_rating) as avg_rating
          FROM 
          (SELECT books.book_id, books.author_id, subq.num_of_ratings, subq.avg_rating, author  
          FROM books
          RIGHT JOIN 
              (SELECT * 
              FROM
              (SELECT book_id, COUNT(rating_id) as num_of_ratings, AVG(rating) as avg_rating
              FROM ratings
              GROUP BY book_id) as subq
              WHERE num_of_ratings >=50
              ORDER BY book_id) as subq on subq.book_id = books.book_id
          LEFT JOIN authors on authors.author_id = books.author_id) as subq2
          GROUP BY author
          ORDER BY avg_rating DESC
          LIMIT 10;
          '''
task_4 = pd.io.sql.read_sql(query_task_4, con = engine) 
task_4

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


In [14]:
#Код для проверки
# books_50 = ratings.groupby('book_id').agg({'rating_id':'count', 'rating':'mean'}).query('rating_id >= 50')
# books_50 = books_50.rename(columns={'rating_id':'num_of_ratings', 'rating':'avg_rating'})
# authors_50 = books.query('book_id in @books_50.index')[['book_id', 'author_id']]

# authors_rating = books_50.merge(authors_50, on='book_id').merge(authors, on='author_id', how='left')\
#                                               .groupby('author').agg({'avg_rating':'mean'})
# authors_rating[authors_rating.avg_rating == authors_rating.avg_rating.max()]

## Задание 5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок<a id="step2.5"></a>   

In [15]:
query_task_5 = '''
          SELECT username, COUNT(review_id)
          FROM reviews
          WHERE username in (
          SELECT username 
              FROM
              (SELECT username, COUNT(rating_id) as num_of_ratings
              FROM ratings
              GROUP BY username) as subq
              WHERE num_of_ratings > 50)
          GROUP BY username
          ORDER BY count
          '''
task_5 = pd.io.sql.read_sql(query_task_5, con = engine) 
task_5

,username,count
0,xdavis,18
1,paul88,22
2,jennifermiller,25
3,richard89,26
4,martinadam,27
5,sfitzgerald,28


In [16]:
#Код для проверки
# user_50 = ratings.groupby('username').agg({'rating_id':'count'}).query('rating_id > 50').index
# reviews.query('username in @user_50').groupby('username').agg({'review_id':'count'}).sort_values(by='review_id')